# 필요 함수 import

In [1]:
import requests
from bs4 import BeautifulSoup
import re
import pandas as pd
import numpy as np

# 1차 : 쿠팡 상품 리스트 추출
> categoryId
- 밀키트 : 502482
- 1국탕전골 : 502483
- 2덮밥/비빔밥 : 502484
- 3스테이크/고기 502485
- 4면/파스타/감바스 502486
- 5분식 502487
- 6중식요리 502490
- 7기타요리 502491

In [2]:
# 쿠팡 페이지 호출 함수
def get_coupang_item(URL, user_agt):
    headers = {"User-Agent":user_agt
               , "Accept-Language": "ko-KR,ko;q=0.8,en-US;q=0.5,en;q=0.3"}
    res = requests.get(URL, headers=headers)
    res.raise_for_status()
    soup = BeautifulSoup(res.text, 'lxml')
    return soup

## 필요 정보만 추출
- 카테고리 대분류, 상품id, 상품only, 상품명(전체), 정가, 할인율, 판매가, 100g당_가격, 별점, 리뷰수, 품절여부

In [3]:
#카테고리 대분류 라벨링 함수
def category_number(categoryId, category):
    return category.get(categoryId)


#해당 카테고리의 페이지 전체 수 가져오기
def page_number(categoryId, user_agt):
    URL = 'https://www.coupang.com/np/categories/'+categoryId+"?page=1"
    cg_soup_base = get_coupang_item(URL, user_agt)
    page_all = cg_soup_base.select('#product-list-paging > div > a')
    page_list = []
    for page_num in page_all:
        page_num = page_num.text
        try:
            page_num = int(page_num)
            page_list.append(page_num)
        except:
            pass
    return page_list[-1]


#1차 쿠팡 상품 리스트 전처리 함수
def p_list_preprocessing(cg_soup, category_num):
    #상품id, data-item-id, data-vendor-item-id
    bady_pdt = cg_soup.select_one("a.baby-product-link")
    productId = bady_pdt['data-product-id']
    itemsId = bady_pdt['data-item-id']
    vendorItemId = bady_pdt['data-vendor-item-id']

    # 상품명
    name = cg_soup.select_one("div.name").text.replace('\n', '').strip()
    name_only = name.split(', ')[0]

    # 판매가
    price_value = cg_soup.select_one("strong.price-value").text.replace(',','')
    
    try:
        # 정가
        base_price = cg_soup.select_one("del.base-price").text.replace("\n",'').strip().replace(',','')
        # 할인율
        discount_pcg = cg_soup.select_one("span.discount-percentage").text
    except:
        base_price, discount_pcg = price_value, 0

    #100g당 가격
    try:
        unit_price = cg_soup.select_one("span.unit-price").text.replace("\n", '').strip()
    except:
        unit_price = np.NaN
    
    try:
        # 별점
        star = cg_soup.select_one("span.star > em").text
        # 리뷰수
        rating_count = cg_soup.select_one("span.rating-total-count").text
        rating_count = re.sub(r'[^0-9]','',rating_count)
    except:
        star, rating_count = 0, 0

    # 품절여부 > 품절이면 1, 아니면 0
    try:
        out_of_stock = cg_soup.select_one("div.out-of-stock").text.replace("\n", '').strip()
        out_of_stock = 1
    except:
        out_of_stock = 0
    
    #DataFrame으로 병합
    coupang_item = pd.DataFrame({'카테고리명':category_num,'상품id':productId, 'data-item-id':itemsId, 'data-vendor-item-id':vendorItemId
                                , '상품':name_only,'상품명':name, '정가':base_price, '할인율':discount_pcg, '판매가':price_value
                                , '100g당_가격':unit_price, '별점':star, '리뷰수':rating_count, '품절여부':out_of_stock}
                                , index=[0])
    return coupang_item

## 쿠팡 상품 리스트 추출 실행부
> 밀키트 추가 필터링 후 전체 크롤링

- 추가 전처리 해야하는 부분
1. 100g당 가격 결측치 처리
2. 구성정보 텍스트 전처리 : 명사 추출 후 재료만 남기기
3. 구성정보 결측치 채우기 : 텍스트로 구성정보가 존재하는 경우만 >> 그외 결측치 삭제
4. 상품명에서 음식명만 추출 > 세부 카테고리 정리

In [4]:
#categoryId 기반 카테고리 대분류 정의
category = {'502483':1, '502484':2, '502485':3, '502486':4, '502487':5, '502490':6, '502491':7}
category_keys = list(category.keys())

#상품 리스트를 담을 빈 dataframe 생성
item_list = ['카테고리명','상품id','data-item-id','data-vendor-item-id','상품','상품명','정가',
             '할인율','판매가','100g당_가격','별점','리뷰수','품절여부']
coupang_items = pd.DataFrame(columns=item_list)

#쿠팡 상품 리스트 추출
cg_url = 'https://www.coupang.com/np/categories/'
user_agt = "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/108.0.0.0 Safari/537.36 Edg/108.0.1462.46"

for categoryId in category_keys:
    page_num = page_number(categoryId, user_agt) #해당 카테고리의 페이지 마지막 페이지 번호 추출
    for page in range(1, page_num+1):
        URL = cg_url+categoryId+"?page="+str(page)+'&filter=1%23attr_12406%2419087%40DEFAULT'
        
        #페이지 호출
        cg_soup_base = get_coupang_item(URL, user_agt)
        cg_soup2 = cg_soup_base.select("#productList li")  

        #상품 리스트 추출 및 전처리
        for cg_soup in cg_soup2:
            coupang_item = p_list_preprocessing(cg_soup, category_number(categoryId, category))
            coupang_items = pd.concat([coupang_items,coupang_item], ignore_index=True)

coupang_items.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 371 entries, 0 to 370
Data columns (total 13 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   카테고리명                371 non-null    object
 1   상품id                 371 non-null    object
 2   data-item-id         371 non-null    object
 3   data-vendor-item-id  371 non-null    object
 4   상품                   371 non-null    object
 5   상품명                  371 non-null    object
 6   정가                   371 non-null    object
 7   할인율                  371 non-null    object
 8   판매가                  371 non-null    object
 9   100g당_가격             328 non-null    object
 10  별점                   371 non-null    object
 11  리뷰수                  371 non-null    object
 12  품절여부                 371 non-null    object
dtypes: object(13)
memory usage: 37.8+ KB


In [5]:
#전체 중복값 확인
coupang_items.loc[coupang_items.duplicated(subset=['상품id']), :]

,카테고리명,상품id,data-item-id,data-vendor-item-id,상품,상품명,정가,할인율,판매가,100g당_가격,별점,리뷰수,품절여부


## '100g당_가격' 결측치 채우기

In [6]:
#결측치 확인
coupang_items.loc[coupang_items['100g당_가격'].isnull()]

,카테고리명,상품id,data-item-id,data-vendor-item-id,상품,상품명,정가,할인율,판매가,100g당_가격,별점,리뷰수,품절여부
12,1,6617266915,15030233120,82252852910,복선당 규스지 사골 도가니탕 2인분 1160g + 소스 20g x 2p 세트 (냉동),복선당 규스지 사골 도가니탕 2인분 1160g + 소스 20g x 2p 세트 (냉동...,20250,9%,18280,NaN,4.5,163,0
21,1,6638341720,15165310214,82386704735,바다자리 모둠조개탕 밀키트 2~3인분 550g,"바다자리 모둠조개탕 밀키트 2~3인분 550g, 1세트",16200,14%,13900,NaN,4.5,433,0
36,1,6908772086,16642374293,83826518677,바다자리 생굴 해물 굴전골 씨키트 3~4인분 680g + 야채 + 소스 세트,"바다자리 생굴 해물 굴전골 씨키트 3~4인분 680g + 야채 + 소스 세트, 1세트",27200,14%,23300,NaN,4.5,9,0
38,1,6909265774,16645168605,83829261640,바다자리 겨울엔 생물 대구 맑은탕 씨키트 2~3인분 700g + 야채,"바다자리 겨울엔 생물 대구 맑은탕 씨키트 2~3인분 700g + 야채, 1세트",34900,13%,30300,NaN,4.5,11,0
42,1,6909265763,16645168518,83829261628,바다자리 생물 아귀 매운탕 씨키트 2~3인분 1kg + 야채,"바다자리 생물 아귀 매운탕 씨키트 2~3인분 1kg + 야채, 1세트",27000,12%,23600,NaN,5.0,4,0
43,1,6908771815,16642373027,83826517085,바다자리 국산갈치구이 앤 해물된장찌개 650g + 야채 + 소스 씨키트 3~4인분 세트,바다자리 국산갈치구이 앤 해물된장찌개 650g + 야채 + 소스 씨키트 3~4인분 ...,22800,14%,19600,NaN,5.0,1,0
51,1,6919308405,16703146452,83885677058,바다자리 고등어구이 앤 생물홍합짬뽕탕 씨키트 2~3인분 680g + 야채,"바다자리 고등어구이 앤 생물홍합짬뽕탕 씨키트 2~3인분 680g + 야채, 1세트",16900,11%,15000,NaN,5.0,1,0
52,1,6624629659,15076284318,82298504346,바다자리 생물 바지락탕용 씨키트 2~3인분 500g,"바다자리 생물 바지락탕용 씨키트 2~3인분 500g, 1세트",14200,19%,11400,NaN,4.5,195,0
56,1,6909266222,16645171576,83829265080,바다자리 생물 아귀 맑은탕 씨키트 2~3인분 900g + 야채,"바다자리 생물 아귀 맑은탕 씨키트 2~3인분 900g + 야채, 1세트",27000,12%,23600,NaN,4.5,15,0
76,1,6909265771,16645168567,83829261659,바다자리 겨울엔 생물 대구 매운탕 씨키트 2~3인분 800g + 야채,"바다자리 겨울엔 생물 대구 매운탕 씨키트 2~3인분 800g + 야채, 1세트",34900,13%,30300,NaN,4.5,6,0


In [7]:
product_url = 'https://www.coupang.com/vp/products/'
productId = 6638341720
URL = product_url+str(productId)
soup_100 = get_coupang_item(URL, user_agt)
soup_100

<!DOCTYPE html>
<!--[if lte IE 7 ]><html class="lt-ie9 lt-ie8" lang="ko-KR"><![endif]--><!--[if IE 8 ]><html class="lt-ie9" lang="ko-KR"><![endif]--><!--[if (gte IE 9)|!(IE)]><!--><html lang="ko-KR"><!--<![endif]-->
<head>
<meta charset="utf-8"/>
<meta content="zaNrGtrOLMjglkziY2IvmL8dOXyCWHGArDHqFazJQVI" name="google-site-verification"/>
<meta content="on" http-equiv="x-dns-prefetch-control"/>
<link href="//cart.coupang.com" rel="dns-prefetch"/>
<link href="//assets.coupang.com" rel="dns-prefetch"/>
<link href="//assets2.coupang.com" rel="dns-prefetch"/>
<link href="//assets.coupangcdn.com" rel="dns-prefetch"/>
<link href="//asset1.coupangcdn.com" rel="dns-prefetch"/>
<link href="//private.coupang.com" rel="dns-prefetch"/>
<link href="//img1a.coupangcdn.com" rel="dns-prefetch"/>
<link href="//image1.coupangcdn.com" rel="dns-prefetch"/>
<link href="//thumbnail1.coupangcdn.com" rel="dns-prefetch"/>
<link href="//static.coupangcdn.com" rel="dns-prefetch"/>
<link href="//www.facebook.com"

# 2차 : 구성 정보 추출
- 메뉴이름, 구성정보

In [12]:
#구성정보 추출 함수
def p_c_preprocessing(productId, p_c_page):
    product_composition = '0'
    for item in p_c_page:
        s = item.text.replace('\n','').strip()
        if product_composition == '2': #3차: 재료부분만 추출
            product_composition = s
        elif product_composition =='1':#2차: 불필요한 text 제외
            product_composition='2'
        elif '구성 정보' in s :         #1차: '구성 정보' 존재여부 확인
            product_composition = '1'
        
        #1-3차 외 다른 위치의 '구성 정보' 추출
        elif '구성과 단위' in s:
            product_composition = s
        elif '로 구성' in s:
            product_composition = s

    #4차: 없으면 결측값 처리
    if product_composition.isdigit():
        product_composition = np.NaN
    
    data_item = pd.DataFrame({'상품id':productId, '구성정보':product_composition}, index=[0])
    return data_item

In [13]:
#2차 실행부
data_items = pd.DataFrame(columns=['상품id', '구성정보'])

for idx in range(len(coupang_items)):
    #각 상품 상세페이지 호출
    product_url = 'https://www.coupang.com/vp/products/'
    productId = coupang_items.loc[idx, '상품id'] #data-product-id
    itemsId = coupang_items.loc[idx, 'data-item-id']  #data-item-id
    vendorItemId = coupang_items.loc[idx, 'data-vendor-item-id'] #data-vendor-item-id
    URL = product_url+str(productId)+'/items/'+str(itemsId)+'/vendoritems/'+str(vendorItemId)
    
    pdt_soup = get_coupang_item(URL, user_agt)
    
    #'구성정보' 추출
    p_c_page = pdt_soup.find('body').children
    data_item = p_c_preprocessing(productId, p_c_page)
    data_items = pd.concat([data_items, data_item], ignore_index=True)

data_items.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 371 entries, 0 to 370
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   상품id    371 non-null    object
 1   구성정보    338 non-null    object
dtypes: object(2)
memory usage: 5.9+ KB


In [14]:
#쿠팡 상품 리스트와 구성정보 병합
coupang = pd.merge(coupang_items, data_items)
coupang.to_csv('coupang_filtering.csv')
coupang.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 371 entries, 0 to 370
Data columns (total 14 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   카테고리명                371 non-null    object
 1   상품id                 371 non-null    object
 2   data-item-id         371 non-null    object
 3   data-vendor-item-id  371 non-null    object
 4   상품                   371 non-null    object
 5   상품명                  371 non-null    object
 6   정가                   371 non-null    object
 7   할인율                  371 non-null    object
 8   판매가                  371 non-null    object
 9   100g당_가격             328 non-null    object
 10  별점                   371 non-null    object
 11  리뷰수                  371 non-null    object
 12  품절여부                 371 non-null    object
 13  구성정보                 338 non-null    object
dtypes: object(14)
memory usage: 43.5+ KB


In [15]:
#구성정보 결측치 확인
print('결측치 개수: ',coupang.구성정보.isnull().sum())
coupang.loc[coupang['구성정보'].isnull()]

결측치 개수:  33


,카테고리명,상품id,data-item-id,data-vendor-item-id,상품,상품명,정가,할인율,판매가,100g당_가격,별점,리뷰수,품절여부,구성정보
3,1,1637283332,2792368476,70782064297,곰곰 우삼겹 순두부 찌개,"곰곰 우삼겹 순두부 찌개, 710g, 1세트",8190,2%,7990,"(100g당 1,125원)",4.5,14198,0,NaN
5,1,4926044090,6464437521,73758857275,곰곰 밀푀유 나베,"곰곰 밀푀유 나베, 1.2kg, 1세트",14990,0,14990,"(100g당 1,249원)",4.5,15582,0,NaN
7,1,1946869683,3305331651,71292240570,곰곰 애호박 된장찌개 밀키트,"곰곰 애호박 된장찌개 밀키트, 730g, 1개",8170,2%,7990,"(100g당 1,095원)",4.5,6463,0,NaN
10,1,2251401014,3848979135,71833775592,곰곰 간편한 청국장찌개 밀키트,"곰곰 간편한 청국장찌개 밀키트, 770g, 1개",11180,0,11180,"(100g당 1,452원)",4.5,4539,0,NaN
13,1,5026557256,6745912116,74038795689,곰곰 소고기 된장찌개 밀키트,"곰곰 소고기 된장찌개 밀키트, 610g, 1개",7900,0,7900,"(100g당 1,295원)",5.0,4694,0,NaN
19,1,2322032858,4010406015,71994624450,곰곰 옛날식 부대찌개,"곰곰 옛날식 부대찌개, 878g, 1개",14900,0,14900,"(100g당 1,697원)",4.5,5773,0,NaN
20,1,1421897672,2460513791,70454023419,프레시밀 감자 짜글이,"프레시밀 감자 짜글이, 770g, 1팩",12500,0,12500,"(100g당 1,623원)",4.5,1387,0,NaN
25,1,4841176792,6256834516,73552540510,곰곰 샤브 버섯 국수 전골 (냉장),"곰곰 샤브 버섯 국수 전골 (냉장), 1.01kg, 1개",12900,10%,11490,"(100g당 1,138원)",4.5,1650,0,NaN
46,1,1717572551,2923204602,70911802212,프레시지 더큰 우삼겹 순두부찌개,"프레시지 더큰 우삼겹 순두부찌개, 1215g, 1개",13900,0,13900,"(100g당 1,144원)",4.5,7380,0,NaN
90,1,6439578445,13933872805,81183220011,딜리조이 마라탕 밀키트,"딜리조이 마라탕 밀키트, 1개",15430,2%,14990,NaN,4.5,351,0,NaN


# 상세페이지 이미지에서 구성정보 추출
- 텍스트 추출 가능한 브랜드 : 곰곰, 딜리조이, 프렙

In [16]:
# 상품 상세페이지 - 상세정보 이미지만 링크 추출 코드
def get_coupang_image(img_URL, user_agt):
    headers = {"User-Agent":user_agt
               , "Accept-Language": "ko-KR,ko;q=0.8,en-US;q=0.5,en;q=0.3"}
    page = requests.get(img_URL, headers= headers).text
    #coupang_image_soup = BeautifulSoup(page.text, "html.parser")
    #return coupang_image_soup
    return page

In [17]:
image_link = []

image_productId = list(coupang.loc[coupang['구성정보'].isnull()]['상품id'].values)
image_dataitemId = list(coupang.loc[coupang['구성정보'].isnull()]['data-item-id'].values)
image_datavendorId = list(coupang.loc[coupang['구성정보'].isnull()]['data-vendor-item-id'].values)

for product in range(len(image_productId)):
    img_URL = product_url + str(image_productId[product]) + '/items/' +str(image_dataitemId[product])+'/vendoritems/'+str(image_datavendorId[product])
    user_agt = "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/108.0.0.0 Safari/537.36 Edg/108.0.1462.46"

    page = get_coupang_image(img_URL, user_agt)

    link_start = []
    for text in re.finditer('//thumbnail',page):
        link_start.append(text.start())
        
    link_end = []
    for text in re.finditer('g"}],',page):
        link_end.append(text.start())
    
    for num in range(len(link_start)):
        image_link.append('https:' + page[int(link_start[num]):int(link_end[num]) + 1])
        
# 뷰솝 파일 구성이 이상해서 html을 텍스트로 불러와서 추출하는 무식한 방법인데.. 
# 더 간단한 방법 있으면 바꿔주세요

In [18]:
# 이미지 다운로드 코드
import urllib.request
count = 0
for i in image_link:
    count += 1
    savename = "저장할이미지이름" + str(count) + ".jpg"

    # url이 가리키는 주소에 접근해서 해당 자원을 로컬 컴퓨터에 저장하기
    urllib.request.urlretrieve(i, savename)

print("저장완료!!!")
# 우선 전체 결측값 링크 넣어봤더니 217개의 이미지 나옴
# 각 링크별로 이미지가 평균 6-7개정도 있는데 구별 용이하게 따로 리스트로 묶어주던지, 
# 곰곰, 딜리조이, 프렙만 추출하는 코드로 전처리 다시 하면 될듯

저장완료!!!


In [19]:
# 이미지에서 텍스트 추출 코드 with pytesserect

# pytesseract github에서 설치하고, 설치 폴더 경로 환경변수에 추가해줘야 합니다. 
# 설치할때 language 에서 Korean 체크 해야함

# pip install pytesseract
# pip install opencv-python (cv2 모듈)

import pytesseract
import cv2 

In [21]:
file = 'testimage1.jpg'
image = cv2.imread(file)
rgb_image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

# 영어 + 한글 혼합된 텍스트 추출
text = pytesseract.image_to_string(rgb_image, lang='kor+eng')
print(text)

error: OpenCV(4.6.0) D:\a\opencv-python\opencv-python\opencv\modules\imgproc\src\color.cpp:182: error: (-215:Assertion failed) !_src.empty() in function 'cv::cvtColor'


### 결과
Better quality / Lower price

Exclusive ,

곰곰의쉽고간편하게
만나는 즐거운요리시간
간편하게손질된재료와정량화된
레시피로만나는 따듯한한끼